In [ ]:
#Imports

import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [ ]:
#Load and prepare data

point_cloud_path = "/home/chris/Code/PointClouds/data/ply/Cover/CoverCleaned.ply"
output_path = ''

pcd = o3d.io.read_point_cloud(point_cloud_path)
xyz_only = np.asarray(pcd.points)  # Extract only x, y, z coordinates
pcd.points = o3d.utility.Vector3dVector(xyz_only)
print(f'Number of points = {len(pcd.points)}')

In [ ]:
#Downsample if necessary

voxel_down_pcd = pcd.voxel_down_sample(voxel_size=1)
xyz_downsampled = np.asarray(voxel_down_pcd.points)
print(f'Number of points in downsampled pcd = {len(voxel_down_pcd.points)}')

In [ ]:
# Visualize point cloud using Plotly
fig = go.Figure(data=[go.Scatter3d(
    x=xyz_downsampled[:, 0],
    y=xyz_downsampled[:, 1],
    z=xyz_downsampled[:, 2],
    mode='markers',
    marker=dict(
        size=2,
        color='blue',
    )
)])

# Set equal aspect ratio and update layout
fig.update_layout(
    scene=dict(
        xaxis_title='X Axis',
        yaxis_title='Y Axis',
        zaxis_title='Z Axis',
        aspectmode='data'  # This forces equal aspect ratio
    ),
    margin=dict(l=0, r=0, b=0, t=0)
)
fig.show()

In [ ]:
#Normal Estimation
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

In [ ]:
#Mesh Generation with Ball Pivoting

distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 3* avg_dist

#Compute the mesh
bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd,o3d.utility.DoubleVector([radius, radius*2]))

dec_mesh = bpa_mesh.simplify_quadric_decimation(100000)


In [ ]:
dec_mesh.remove_degenerate_triangles()
dec_mesh.remove_duplicated_triangles()
dec_mesh.remove_duplicated_vertices()
dec_mesh.remove_non_manifold_edges()

In [ ]:
output_mesh_path = "/home/chris/Code/PointClouds/data/obj/ConvertedMesh.obj"  # Change this to your desired path

# Save the mesh as an OBJ file
o3d.io.write_triangle_mesh(output_mesh_path, dec_mesh)